In [6]:
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt
import pandas as pd
import os


In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", None, "display.max_columns",
              None, 'display.max_colwidth', None)


In [8]:
path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')
path_result = os.path.join(os.path.dirname(os.getcwd()), 'Result')

path_challenge = os.path.join(path_result, 'Challenge')
path_challenge_evolution = os.path.join(path_challenge, 'Evolution')

path_solution = os.path.join(path_result, 'Solution')
path_solution_evolution = os.path.join(path_solution, 'Evolution')

In [9]:
# combine issues and questions

df_issues = pd.read_json(os.path.join(path_dataset, 'issues.json'))
df_questions = pd.read_json(os.path.join(path_dataset, 'questions.json'))

df_issues['Challenge_link'] = df_issues['Issue_link']
df_issues['Challenge_original_content'] = df_issues['Issue_original_content']
df_issues['Challenge_preprocessed_content'] = df_issues['Issue_preprocessed_content']
df_issues['Challenge_summary'] = df_issues['Issue_gpt_summary']
df_issues['Challenge_creation_time'] = df_issues['Issue_creation_time']
df_issues['Challenge_comment_count'] = df_issues['Issue_comment_count']
df_issues['Challenge_score'] = df_issues['Issue_upvote_count'] - df_issues['Issue_downvote_count']
df_issues['Challenge_closed_time'] = df_issues['Issue_closed_time']
df_issues['Solution_summary'] = df_issues['Fix_manual_summary']

df_questions['Challenge_link'] = df_questions['Question_link']
df_questions['Challenge_original_content'] = df_questions['Question_original_content']
df_questions['Challenge_preprocessed_content'] = df_questions['Question_preprocessed_content']
df_questions['Challenge_summary'] = df_questions['Question_gpt_summary']
df_questions['Challenge_creation_time'] = df_questions['Question_creation_time']
df_questions['Challenge_comment_count'] = df_questions['Question_comment_count']
df_questions['Challenge_score'] = df_questions['Question_score']
df_questions['Challenge_closed_time'] = df_questions['Question_closed_time']
df_questions['Solution_original_content'] = df_questions['Answer_original_content']
df_questions['Solution_preprocessed_content'] = df_questions['Answer_preprocessed_content']
df_questions['Solution_summary'] = df_questions['Answer_gpt_summary']

del df_issues['Issue_title']
del df_issues['Issue_body']
del df_issues['Issue_link']
del df_issues['Issue_creation_time']
del df_issues['Issue_comment_count']
del df_issues['Issue_upvote_count']
del df_issues['Issue_downvote_count']
del df_issues['Issue_original_content']
del df_issues['Issue_preprocessed_content']
del df_issues['Issue_gpt_summary_original']
del df_issues['Issue_gpt_summary']
del df_issues['Issue_closed_time']
del df_issues['Fix_manual_summary_original']
del df_issues['Fix_manual_summary']

del df_questions['Question_title']
del df_questions['Question_body']
del df_questions['Question_link']
del df_questions['Question_creation_time']
del df_questions['Question_comment_count']
del df_questions['Question_score']
del df_questions['Question_original_content']
del df_questions['Question_preprocessed_content']
del df_questions['Question_gpt_summary_original']
del df_questions['Question_gpt_summary']
del df_questions['Question_closed_time']
del df_questions['Answer_original_content']
del df_questions['Answer_preprocessed_content']
del df_questions['Answer_gpt_summary_original']
del df_questions['Answer_gpt_summary']

df_all = pd.concat([df_issues, df_questions], ignore_index=True)
df_all.to_json(os.path.join(path_dataset, 'original.json'),
               indent=4, orient='records')

In [10]:
# remove custom stop words from challenges and solutions

from gensim.parsing.preprocessing import remove_stopwords

stop_words_custom = [
    'amazon',
    'answer',
    'appreci',
    'attempt',
    'azur',
    'challeng',
    'deep learn',
    'differ',
    'difficulti',
    'encount',
    'error',
    'experi',
    'databrick',
    'face',
    'fix',
    'googl',
    'help',
    'includ',
    'issu',
    'look',
    'machin learn',
    'microsoft',
    'need',
    'possibl',
    'problem',
    'project',
    'provid',
    'python',
    'pytorch',
    'question',
    'resolv',
    'respond',
    'seek',
    'solut',
    'solv',
    'struggl',
    'suggest',
    'tensorflow',
    'thank',
    'try',
    'unabl',
    'us',
    'user'
]

df_all = pd.read_json(os.path.join(path_dataset, 'original.json'))

for index, row in df_all.iterrows():
    df_all.at[index, 'Challenge_original_content'] = remove_stopwords(row['Challenge_original_content'].replace(
        'Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
    df_all.at[index, 'Challenge_preprocessed_content'] = remove_stopwords(
        row['Challenge_preprocessed_content'].replace('Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
    df_all.at[index, 'Challenge_summary'] = remove_stopwords(row['Challenge_summary'], stopwords=stop_words_custom)

    if row['Solution_original_content']:
        df_all.at[index, 'Solution_original_content'] = remove_stopwords(row['Solution_original_content'].replace(
            'Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
        df_all.at[index, 'Solution_preprocessed_content'] = remove_stopwords(
            row['Solution_preprocessed_content'].replace('Title: ', '').replace('Answer: ', ''), stopwords=stop_words_custom)
        df_all.at[index, 'Solution_summary'] = remove_stopwords(row['Solution_summary'], stopwords=stop_words_custom)

df_all.to_json(os.path.join(path_dataset, 'preprocessed.json'),
               indent=4, orient='records')

In [11]:
df_all = pd.read_json(os.path.join(path_dataset, 'preprocessed.json'))

# remove issues with uninformed content
for index, row in df_all.iterrows():
    if len(row['Challenge_original_content'].split()) < 6 or len(row['Challenge_original_content']) < 30:
        print(row['Challenge_original_content'])
        df_all.drop(index, inplace=True)
    elif row['Solution_original_content'] and (len(row['Solution_original_content'].split()) < 6 or len(row['Solution_original_content']) < 30):
        print(row['Solution_original_content'])
        df_all.drop(index, inplace=True)

df_all.to_json(os.path.join(path_dataset, 'filtered.json'),
               indent=4, orient='records')

wonder initi studio lab titl
modulenotfounderror modul tensorboard
combin param param work
access allow access member given
load
deploy fail
log val loss
import
tensorboard default logger option
logger
run end run finish creat
vpc endpoint api api st
support hive adl
cloudform templat sure
csv file folder recordio
littl involv ye
file pin azurestor
processingstep uri code locat refer
leav case come accross
pipelin moment
aw cli aw sdk exampl
let know happen
delet creat endpoint
report appear bug team investig
code work
like templat run creat instanc
passau refer visual studio magazin
bug got correct todai close
upgrad sqlalchemi
file path ensur correct path
tri work let file transient
think chang log stream servic
situat chang git avail
sdk support automl sdk support
actual come smdebug version downgrad
updat core
forc gpu devic devic gpu
begin list
accord document maximum
sure estim framework version version
web termin import delet
ye maxtrialscallback exact featur situat
add run flus

In [68]:
df_all = pd.read_json(os.path.join(path_dataset, 'topics.json'))

df_challenge = df_all.groupby('Challenge_topic').agg(Mean_score=('Challenge_score', 'mean'), Mean_comments=('Challenge_comment_count', 'mean'), Count = ('Challenge_topic', 'count')).reset_index().round(2)
df_challenge['Score_comment_ratio'] = (df_challenge['Mean_score'] / df_challenge['Mean_comments']).round(2)
df_challenge = df_challenge.sort_values('Score_comment_ratio', ascending=False)
df_challenge.set_index('Challenge_topic')


,Mean_score,Mean_comments,Count,Score_comment_ratio
Challenge_topic,,,,
35,1.15,0.18,33,6.39
33,0.97,0.25,38,3.88
18,2.14,0.90,99,2.38
1,0.59,0.28,274,2.11
28,1.22,0.62,41,1.97
41,0.69,0.36,16,1.92
31,1.27,0.69,40,1.84
10,1.65,0.92,142,1.79
37,0.97,0.56,71,1.73


In [ ]:
df_challenge = df_challenge.sort_values('Mean_score', ascending=False)
df_challenge[['Mean_score', 'Mean_comments']].plot(kind='bar', figsize=(15, 8), title='Challenge topics')

In [12]:
# Explore challenge topics evolution

df_all = pd.read_json(os.path.join(path_dataset, 'topics.json'))

# BigQuery Stack Overflow public dataset is updated until Nov 24, 2022, 1:39:22 PM UTC-5
df_challenge = df_all[df_all['Challenge_creation_time'] < '2022-11-21']
    
for name, group in df_challenge.groupby('Challenge_topic'):
    group = group.groupby(pd.Grouper(key='Challenge_creation_time', freq='W')).agg(Count = ('Challenge_topic', 'count')).reset_index()
    fig = plt.figure()
    axis = fig.add_subplot(111)
    group.plot(x='Challenge_creation_time',y='Count',ax=axis, grid=True, color='red')
    axis.axes.get_xaxis().get_label().set_visible(False) 
    axis.set_ylabel('Topic evolution', fontsize=20)
    plt.title(str(name), fontsize=30)
    fig.autofmt_xdate()
    fig.savefig(os.path.join(path_challenge_evolution, f'Topic_{name}'), bbox_inches="tight")
    plt.close()


In [ ]:
# Explore solution topics evolution

df_all = pd.read_json(os.path.join(path_dataset, 'topics.json'))

# BigQuery Stack Overflow public dataset is updated until Nov 24, 2022, 1:39:22 PM UTC-5
df_solution = df_all[df_all['Solution_topic'] > -1]
df_solution = df_solution[df_solution['Challenge_creation_time'] < '2022-11-21']

for name, group in df_solution.groupby('Challenge_topic'):
    group = group.groupby(pd.Grouper(key='Solution_creation_time', freq='W')).agg(Count = ('Challenge_topic', 'count')).reset_index()
    fig = plt.figure()
    axis = fig.add_subplot(111)
    group.plot(x='Solution_creation_time',y='Count',ax=axis, grid=True, color='red')
    axis.axes.get_xaxis().get_label().set_visible(False) 
    axis.set_ylabel('Topic evolution', fontsize=20)
    plt.title(str(name), fontsize=30)
    fig.autofmt_xdate()
    fig.savefig(os.path.join(path_solution_evolution, f'Topic_{name}'), bbox_inches="tight")
    plt.close()


In [ ]:
# average answer time
df_all['Answer_time'] = df_all['Solution_creation_time'] - df_all['Challenge_creation_time']